# Image Detection Model Testing
This notebook will test image detection models against captured screenshots. I plan to validate the 3 models below:
1. https://universe.roboflow.com/d41/attack-shv5h
2. https://universe.roboflow.com/workspace-sgzwp/diablo4_test2
3. https://universe.roboflow.com/diablo-4-fojil/diablo-4
